<a href="https://colab.research.google.com/github/CAVASOL/aiffel_quest/blob/main/ML_node/ML_with_Python_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NLP**
* 한국어 문장을 형태소 단위로 분리할 수 있다
* 자연어 전처리
* 긍정 또는 부정의 감성 분석 실행

**ex.**

함께 탐험하며 성장하는 AI 학교 AIFFEL  

형태소 분석 - ['함께', '탐험', '하며', '성장하는', 'AI', '학교', 'AIFFEL']  

어휘 사전 구축(벡터) - {'ai': 0, 'aiffel': 1, '성장하는': 2, '탐험': 3, '하며': 4, '학교': 5, '함께': 6}  

**CountVectorizer** 각 문장에서 단어 출현 횟수를 카운팅 하는 방법 ex, BOW - Bag Of Word  

**TfidfVectorizer** 다른 문서보다 특정 문서에 자주 나타나는 단어에 높은 가중치를 주는 방법  

**형태소분석기**  
* konlpy 한국어 처리를 위한 형태소 분석기  
* 형태소: 의미를 가지는 요소로서 더 이상 분석할 수 없는 가장 작은 단위의 말

In [ ]:
# Ignore warnings
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# i konlpy
!pip install konlpy

In [ ]:
# Lib
import konlpy
from konlpy.tag import Okt
tokenizer = Okt()

In [ ]:
# Tokenization (형태소 단위)
text = "함께 탐험하며 성장하는 AI 학교 AIFFEL"
tokenizer.morphs(text)

In [ ]:
# Tokenization (명사만 추출)
tokenizer.nouns(text)

In [ ]:
# Tokenization (품사 태깅)
tokenizer.pos(text)

**CountVectorizer**

In [ ]:
# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [ ]:
# 단어 토큰화, Okt
words = tokenizer.morphs(text)

In [ ]:
# 데이터 학습
vect.fit(words)

In [ ]:
# 학습된 어휘
vect.get_feature_names_out()

In [ ]:
# 단어 사전
vect.vocabulary_

In [ ]:
# 단어 사전 크기
len(vect.vocabulary_)

In [ ]:
# 인코딩
df_t = vect.transform(words)

In [ ]:
# 인코딩 된 데이터 매트릭스
df_t.toarray()

In [ ]:
# 어휘와 피처 (Dataframe)
pd.DataFrame(df_t.toarray(), columns=vect.get_feature_names())

In [ ]:
# 새로운 단어(데이터)가 추가되면 어떻게 될까?
test = "AI 공부하며 함께 성장해요!"

In [ ]:
# 단어 토큰화 (Okt)
words = tokenizer.morphs(test)
words

In [ ]:
# 인코딩 된 데이터 매트릭스
test_t = vect.transform(words)
test_t.toarray()

In [ ]:
# 어휘와 피처 (Dataframe)
pd.DataFrame(test_t.toarray(), columns=vect.get_feature_names())

**TfidfVectorizer**

In [ ]:
# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# tf-idf 활용 어휘 사전 구축
vect = TfidfVectorizer()
words = tokenizer.morphs(text)
vect.fit(words)
vect.vocabulary_

In [ ]:
# 인코딩 된 데이터 매트릭스
vect.transform(words).toarray()

**감성 분석**

문제정의 > 탐색적 데이터 분석 EDA > *데이터 전처리 > 모델학습 > *예측  

*데이터전처리: 토큰화 > 어휘 사전 구축 > 인코딩  
*예측: 긍정 | 부정  

In [ ]:
# Lib
import pandas as pd

In [ ]:
# Data
df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep="\t")

In [ ]:
# Data sample
df.head()

In [ ]:
# 데이터 크기
df.shape

In [ ]:
# 타겟 확인
df['label'].value_counts()

In [ ]:
# 결측치(Null)
df.isnull().sum()

In [ ]:
# 결측치 삭제
print(df.shape)
df = df.dropna()
print(df.shape)

In [ ]:
# 피처 엔지니어링 (문장의 길이)
df['len'] = df['document'].apply(len)
df.head()

In [ ]:
# len 시각화 (label == 0)
import matplotlib.pyplot as plt
df[df.label==0]['len'].plot(kind='hist')

In [ ]:
# len 시각화 (label == 1)
df[df.label==1]['len'].plot(kind='hist')

In [ ]:
# 데이터 샘플링 df[:000]
df = df[:1000]
df.shape

In [ ]:
# 토큰화
vect = CountVectorizer(tokenizer = tokenizer.morphs)
vectors = vect.fit_transform(df['document'])

**ML 교차검증**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

model = RandomForestClassifier(random_state=2022)
cross_val_score(model, vectors, df['label'], scoring='f1', cv=5).mean()

**한글 NLP**  
* 00% 이상 나타나는 단어는 무시  
* 최소 n개의 문장에만 나타나는 단어 유지  
* 불용어(stopword) 을, 는, 이, 가, 여기, 저기 와 같이 큰 의미가 없는 단어  
* 띄어쓰기  
* 반복되는 글자 정리  
* 맞춤법 검사  

**어휘 사전 구축**

In [ ]:
# 토큰화(max_df) n개 보다 큰 단어 수 무시
vect = CountVectorizer(tokenizer = tokenizer.morphs, max_df=10)
vectors = vect.fit_transform(df['document'])

model = RandomForestClassifier(random_state=2022)
cross_val_score(model, vectors, df['label'], scoring='accuracy', cv=5).mean()

0.643

In [ ]:
# 토큰화(min_df) n개 보다 작은 단어 수 무시
vect = CountVectorizer(tokenizer = tokenizer.morphs, min_df=2)
vectors = vect.fit_transform(df['document'])

model = RandomForestClassifier(random_state=2022)
cross_val_score(model, vectors, df['label'], scoring='accuracy', cv=5).mean()

0.689

**불용어(stopword)**

In [ ]:
# stop_words
text = "함께 탐험하며 성장하는 AI 학교 AIFFEL"
stop_words = ['하며', 'ai']
vect = CountVectorizer(stop_words = stop_words)
words = tokenizer.morphs(text)
vect.fit(words)
vect.vocabulary_

{'함께': 4, '탐험': 2, '성장하는': 1, '학교': 3, 'aiffel': 0}

**띄어쓰기**

In [ ]:
# i spacing
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
# spacing
from pykospacing import Spacing
spacing = Spacing()
text = "함께탐험하며성장하는AI학교AIFFEL"
spacing(text)

'함께 탐험하며 성장하는 AI 학교 AIFFEL'

**반복되는 글자 처리** https://github.com/lovit/soynlp

In [ ]:
# i soynlp
!pip install soynlp

In [ ]:
# 댓글 데이터에 등장하는 반복되는 이모티콘 정리
from soynlp.normalizer import *
emoticon_normalize('ㅋㅋㅋㅋㅋㅋㅋㅋㅋ', num_repeats=3)

'ㅋㅋㅋ'

**맞춤법 검사기** https://github.com/ssut/py-hanspell

In [ ]:
# i py-hanspell
# !pip install git+https://github.com/jungin500/py-hanspell

In [ ]:
# 맞춤법 검사
# from hanspell import spell_checker
# text = "알파고 이전, 2015년부터 만 들 어진 최초의 AI 커뮤니티 모두의연구소.학연,지연,모두연이라는 말이나올만큼

In [ ]:
# 수정된 문장
# result = spell_checker.check(text)

In [ ]:
# result.checked